# This notebook is an example of display of RDI ADCP data: leverages panel

comments:

- use of dimension for `E`, `N`, probably not optimal
- two time dimensions a bit confusing ... at least compute positions along dimension time


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import panel as pn

hv.extension('bokeh')
pn.extension(sizing_mode="stretch_width")

#from pyproj import Geod
#g = Geod(ellps='WGS84')

#import util_adcp
import cswot_adcp.adcp as ad
import cswot_adcp.maps as mp

---
## Load RDI ADCP file (STA: Short Time Averaged)

In [ ]:
file_name = "ADCP_DriX__20220922T202647_018_000000.STA"

In [ ]:
# read data

# STA file reading
STA=ad.read_WH300(file_name)
# navigation compensation
STA=ad.ADCPcompNav(STA)
" / ".join(list(STA))

In [ ]:
def get_extent(STA, buffer=.1):
    """ compute horizontal extent of the STA"""
    lon, lat = STA["elongitude_gps"], STA["elatitude_gps"]
    lon_scale = 1/np.cos(np.pi/180*lat.mean())
    extent = [lon.min() - buffer*lon_scale,
              lon.max() + buffer*lon_scale,
              lat.min() - buffer,
              lat.max() + buffer,
             ]
    extent = [float(e) for e in extent]
    return extent


In [ ]:
extent = get_extent(STA, buffer=.01)

---
## DISPLAY data

### Display current velocity magnitude, direction and correlation

In [ ]:
# override time_gps as it seems to be not reliable
# may introduce inaccurate positions
for v in STA.reset_coords():
    if "time_gps" in STA[v].dims and v!="time_gps":
        STA[v] = STA[v].rename(time_gps="time").assign_coords(time=STA.time)
        
# compute positions at positions time
# not working because of strange gps time values
#STA = STA.assign_coords(longitude=("time", STA["elongitude_gps"].interp(time_gps=STA.time).values),
#                        latitude=("time", STA["elatitude_gps"].interp(time_gps=STA.time).values),
#                       )


In [ ]:
# round time for safety
STA["time"] = STA.time.dt.round("500ms")
STA = STA.assign_coords(time_date = STA.time)
STA["time"] = (STA.time - STA.time[0])/pd.Timedelta("1s")

#t = STA.drop("time_gps").time.dt.round("500ms")
#STA = STA.assign_coords(time=t)
#STA = STA.assign_coords(time=(t - t[0])/pd.Timedelta("1s"))


In [ ]:
frame_slider = pn.widgets.DiscreteSlider(name="time", options=list(STA.time.values), formatter="%0.1f")
# formatter options but complicated with irregular time sampling
frame_slider

In [ ]:
# split velocity field dir dimension into multiple variables 

long_names = dict(E="Eastward velocity", N="Northward velocity", 
                  U="Upward velocity", err="Error velocity",
                  Mag="Velocity magnitude", Dir="Velocity Direction",
                 )
units = dict(E="m/s", N="m/s", U="m/s", err="m/s", Mag="m/s", Dir="degrees")
dirs = STA.dir.values

def split_speed(da):
    ds = xr.merge([da.sel(dir=d).rename(da.name+"_"+str(d))
                   .assign_attrs(units=units[d], long_name=long_names[d]) for d in dirs], 
                  compat="override")
    del ds.attrs["units"]
    return ds.assign_attrs(long_name="compensated velocity fields")

ds_vel = split_speed(STA["vel comp Nav"].rename("compensated"))

In [ ]:
# some dimensions
HEIGHT=200
#WIDTH=1000

In [ ]:
plot_amplitude = (ds_vel["compensated_Mag"]
                  .hvplot(x="time", y="range", responsive=True, clim=(0,1), height=HEIGHT, cmap="inferno")
                  .opts(invert_yaxis=True, title="velocity magnitude")
)

plot_amplitude

In [ ]:
plot_dir = (ds_vel["compensated_Dir"]
                  .hvplot(x="time", y="range", responsive=True, clim=(-180,180), height=HEIGHT, cmap="hsv")
                  .opts(invert_yaxis=True, title="velocity direction")
)

plot_dir

In [ ]:
plot_corr = (STA["corr"].mean("beam")
                  .hvplot(x="time", y="range", responsive=True, clim=(0,100), height=HEIGHT, cmap="hot")
                  .opts(invert_yaxis=True, title="correlation")
)

plot_corr

### vertical profiles

In [ ]:
def get_profile_amplitude(frame):
    return (ds_vel.sel(time=frame)
            .hvplot.line(y="compensated_Mag", x="range", responsive=True, height=HEIGHT, ylim=(0,1))
            .opts(invert_axes=True, invert_yaxis=True, title="velocity magnitude profile")
           )

def get_profile_direction(frame):
    return (ds_vel.sel(time=frame)
            .hvplot.line(y="compensated_Dir", x="range", responsive=True, height=HEIGHT, ylim=(-180,180))
            .opts(invert_axes=True, invert_yaxis=True, title="velocity direction profile")
           )

def get_profile_correlation(frame):
    return (STA.corr.sel(time=frame).mean("beam")
            .hvplot.line(x="range", responsive=True, height=HEIGHT, ylim=(0,200))
            .opts(invert_axes=True, invert_yaxis=True, title="correlation profile")
           )

def get_profile_arrows(frame):
    _ds = ds_vel.sel(time=frame).expand_dims(x=[0]) # np.zeros_like(ds_vel.range.values)
    _ds = _ds.where(_ds.compensated_Mag<1)
    delta = float(_ds["range"].max() - _ds["range"].min())
    _ds["angle"] = _ds["compensated_Dir"]*np.pi/180
    #_ds["mag"] = _ds['compensated_Mag']*delta # rescale amplitude
    _ds["mag"] = _ds['compensated_Mag']*1 # rescale amplitude    
    return _ds.hvplot.vectorfield(x="x", y='range', angle='angle', mag='mag', 
                                  xlim=(-delta/2,delta/2), height=HEIGHT, # hover=False                               
                                 ).opts(magnitude='mag',
                                        color="mag", colorbar=True, clim=(0,1),
                                        invert_yaxis=True, title="velocity",
                                        rescale_lengths=False, scale=10/delta, )

get_profile_arrows(0)


In [ ]:
#ds_vel.hvplot.vectorfield?

### map and trajectory

Available tiles:

['CartoDark', 'CartoEco', 'CartoLight', 'CartoMidnight', 
 'EsriImagery', 'EsriNatGeo', 'EsriReference', 'EsriStreet', 'EsriTerrain', 'EsriUSATopo', 
 'OSM', 'OpenTopoMap', 
 'StamenLabels', 'StamenLabelsRetina', 'StamenTerrain', 'StamenTerrainRetina', 'StamenToner', 
 'StamenTonerBackground', 'StamenTonerBackgroundRetina', 'StamenTonerRetina', 'StamenWatercolor', 
 'Wikipedia']

In [ ]:
def get_trajectory(frame):
    _df = STA[['elongitude_gps', 'elatitude_gps']].to_dataframe()
    base = _df.hvplot.points('elongitude_gps', 'elatitude_gps', geo=True, color='gray', alpha=0.2,
                  xlim=(extent[0], extent[1]), ylim=(extent[2], extent[3]), tiles='EsriNatGeo',
                  #width=500, height=500,
                 )
    focus = _df.loc[[frame]].hvplot.points('elongitude_gps', 'elatitude_gps', geo=True, color='red', alpha=1.,
                 )
    return base*focus

#get_trajectory(0)

### vertical line

In [ ]:
def get_vline(frame):
    return hv.VLine(frame).opts(color="red")
#get_vline(1)

### title bar

In [ ]:
# 
app_bar = pn.Row(
    pn.pane.Markdown("## STA ADCP data viewer", style={"color": "black"}, width=500, sizing_mode="fixed", margin=(10,5,10,15)), 
    #pn.Spacer(),
    #pn.pane.PNG("http://holoviews.org/_static/logo.png", height=50, sizing_mode="fixed", align="center"),
    #pn.pane.PNG("https://panel.holoviz.org/_static/logo_horizontal.png", height=50, sizing_mode="fixed", align="center"),
    background="white",
)
app_bar

### make dynamical plots

In [ ]:
@pn.depends(frame=frame_slider)
def profile_amplitude(frame):
    return get_profile_amplitude(frame)

@pn.depends(frame=frame_slider)
def profile_direction(frame):
    return get_profile_direction(frame)

@pn.depends(frame=frame_slider)
def profile_correlation(frame):
    return get_profile_correlation(frame)

@pn.depends(frame=frame_slider)
def profile_arrows(frame):
    return get_profile_arrows(frame)

@pn.depends(frame=frame_slider)
def trajectory(frame):
    return get_trajectory(frame)

@pn.depends(frame=frame_slider)
def vline(frame):
    return get_vline(frame)

vline_dmap = hv.DynamicMap(vline)
pamplitude_dmap = hv.DynamicMap(profile_amplitude)
pdirection_dmap = hv.DynamicMap(profile_direction)
pcorrelation_dmap = hv.DynamicMap(profile_correlation)
parrows_dmap = hv.DynamicMap(profile_arrows)
trajectory_dmap = hv.DynamicMap(trajectory)

---

### assemble the app

In [ ]:
app = pn.Column(
    app_bar,
    pn.Spacer(height=10),
    pn.Row(trajectory_dmap, 
           parrows_dmap,
          ),
    frame_slider,
    pn.Row(
        plot_amplitude * vline_dmap,
        pamplitude_dmap,
    ),
    pn.Row(
        plot_dir * vline_dmap,
        pdirection_dmap,
    ),
    pn.Row(
        plot_corr * vline_dmap,
        pcorrelation_dmap,
    ),
)
app

In [ ]:
# initial attempt, it is easier to control width

plots = ((plot_amplitude + plot_dir + plot_corr) * vline_dmap).cols(1)

app = pn.Column(
    app_bar,
    pn.Spacer(height=10),
    frame_slider,
    pn.Row(
        plots,
        pn.Column(
            pn.Spacer(height=20),
            pamplitude_dmap,
            pdirection_dmap,
            pcorrelation_dmap,
            width=200,
        ),
        width=1000,
    ),
)
app